# Logarithmic Parameters
This notebook explores Bayesian optimisation of a function who's parameter is best thought of logarithmically (the order of magnitude is more important than the value itself)

To accommodate this, the surrogate function is trained on the exponents of the values rather than the values themselves

note: for this particular function, a $\nu=2.5$ works better for the Matern kernel than $\nu=1.5$.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.debugger import Tracer # debugging
from IPython.display import clear_output
import time

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp

In [ ]:
# local modules
import sys; sys.path.append('../src/')
import optimisation as op
import optimisation_gui as op_gui

In [ ]:
# make deterministic
np.random.seed(101)

Function to optimize:

In [ ]:
f = lambda x: np.cos(2*(20-x)**2)/x - 2*np.log(x)
x_min = 0.005
x_max = 5
xs = op.logspace(x_min, x_max, num_per_mag=200)
#xs = np.linspace(x_min, x_max, num=601)
print(len(xs))
ys = f(xs)
best_y = np.max(ys)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(xs, ys, 'g-')
plt.margins(0.1, 0.1)
plt.title('Linear Scale')
plt.xlabel('x')
plt.ylabel('cost')
plt.show()

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(xs, ys, 'g-')
plt.margins(0.1, 0.1)
plt.title('Logarithmic Scale')
plt.xlabel('x')
plt.axes().set_xscale('log')
plt.ylabel('cost')
plt.show()

In [ ]:
ranges = {
    'x': xs,
    #'y': [5],
    #'z': np.linspace(0, 10, num=10)
}
gp_params = dict(
    alpha = 1e-10, # larger => more noise. Default = 1e-10
    kernel = 1.0 * gp.kernels.Matern(nu=2.5) + gp.kernels.WhiteKernel(),
    #kernel = 1.0 * gp.kernels.RBF(length_scale=1),
    n_restarts_optimizer = 10,
    # make the mean 0 (theoretically a bad thing, see docs, but can help)
    normalize_y = False,
    copy_X_train = True # make a copy of the training data
)

In [ ]:
class MyEvaluator(op.Evaluator):
    def test_config(self, config):
        return f(config.x)

In [ ]:
optimiser = op.BayesianOptimisationOptimiser(ranges,
                                             maximise_cost=True,
                                             acquisition_function='UCB',
                                             acquisition_function_params=dict(kappa=2),
                                             gp_params=gp_params,
                                             pre_samples=3,
                                             ac_max_params=None,
                                             close_tolerance=0.001)
optimiser.step_log_keep = 100
evaluator = MyEvaluator()

In [ ]:
optimiser.run_sequential(evaluator, max_jobs=11)

In [ ]:
optimiser.plot_cost_over_time(true_best=best_y);

In [ ]:
def plot(n, step):
    optimiser.plot_step_slice('x', n, true_cost=f);
op_gui.step_log_slider(optimiser, plot, pre_compute=False);